<a href="https://colab.research.google.com/github/pragyamishra12/KNN-Implementation/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from math import sqrt

from sklearn.preprocessing import StandardScaler

In [ ]:
#upload dataset to colab
from google.colab import files
uploaded = files.upload()

Saving test_genetic_disorders.csv to test_genetic_disorders.csv


In [ ]:
data = (pd.read_csv('test_genetic_disorders.csv'))

data.head()

,Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,...,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5
0,PID0x4175,6.0,No,Yes,No,No,4.981655,Charles,NaN,Kore,...,-99,2.0,Multiple,-99.000000,slightly abnormal,True,True,True,True,True
1,PID0x21f5,10.0,Yes,No,NaN,Yes,5.118890,Catherine,NaN,Homero,...,Yes,-99.0,Multiple,8.179584,normal,False,False,False,True,False
2,PID0x49b8,5.0,No,NaN,No,No,4.876204,James,NaN,Danield,...,No,0.0,Singular,-99.000000,slightly abnormal,False,False,True,True,False
3,PID0x2d97,13.0,No,Yes,Yes,No,4.687767,Brian,NaN,Orville,...,Yes,-99.0,Singular,6.884071,normal,True,False,True,False,True
4,PID0x58da,5.0,No,NaN,NaN,Yes,5.152362,Gary,NaN,Issiah,...,No,-99.0,Multiple,6.195178,normal,True,True,True,True,False


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9463 entries, 0 to 9462
Data columns (total 43 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Id                                        9290 non-null   object 
 1   Patient Age                                       9290 non-null   float64
 2   Genes in mother's side                            9290 non-null   object 
 3   Inherited from father                             8746 non-null   object 
 4   Maternal gene                                     5634 non-null   object 
 5   Paternal gene                                     9290 non-null   object 
 6   Blood cell count (mcL)                            9290 non-null   float64
 7   Patient First Name                                9290 non-null   object 
 8   Family Name                                       147 non-null    object 
 9   Father's name      

In [ ]:
#removing unnecessary features
data = data.drop(columns=['Patient Id','Patient First Name','Family Name','Father\'s name','Institute Name','Location of Institute','Test 1','Test 2','Test 3','Test 4',
'Test 5','Parental consent'])

In [ ]:
#all null rows
data[data.isnull().all(1)].shape

(173, 31)

In [ ]:
#removing all null rows
data = data[data.isnull().all(1)!=True]

data.shape


(9290, 31)

In [ ]:
#rows with missing values
data[data.isnull().any(axis=1)]

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),...,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5
1,10.0,Yes,No,NaN,Yes,5.118890,33.0,53.0,Alive,NaN,...,Yes,-99.0,Multiple,8.179584,normal,False,False,False,True,False
2,5.0,No,NaN,No,No,4.876204,48.0,60.0,Deceased,NaN,...,No,0.0,Singular,-99.000000,slightly abnormal,False,False,True,True,False
4,5.0,No,NaN,NaN,Yes,5.152362,41.0,38.0,Deceased,Tachypnea,...,No,-99.0,Multiple,6.195178,normal,True,True,True,True,False
5,9.0,No,No,NaN,No,4.942384,27.0,25.0,Alive,-99,...,Yes,0.0,-99,6.478252,-99,True,True,True,True,True
6,4.0,Yes,No,NaN,No,5.113778,19.0,22.0,Alive,Normal (30-60),...,No,1.0,-99,3.173918,slightly abnormal,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9283,6.0,No,No,NaN,Yes,4.471051,29.0,25.0,Alive,Normal (30-60),...,No,-99.0,Multiple,6.791320,abnormal,False,True,False,False,False
9284,9.0,No,No,Yes,No,4.763186,30.0,62.0,Deceased,NaN,...,No,0.0,Singular,4.765603,inconclusive,True,True,False,True,False
9286,5.0,Yes,No,NaN,Yes,4.754449,45.0,27.0,Deceased,NaN,...,-99,1.0,Singular,-99.000000,inconclusive,True,True,True,True,False
9287,5.0,Yes,Yes,No,Yes,4.824619,26.0,27.0,Alive,Tachypnea,...,No,2.0,Singular,8.291072,-99,True,True,True,True,True


In [ ]:
for i in data.columns:
    if data[i].dtype.name == 'object':
        data[i] = data[i].fillna(data[i].mode()[0])
    elif data[i].dtype.name != 'object':
        data[i] = data[i].fillna(data[i].mean())

data

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),...,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5
0,6.0,No,Yes,No,No,4.981655,38.0,61.0,Alive,Tachypnea,...,-99,2.0,Multiple,-99.000000,slightly abnormal,True,True,True,True,True
1,10.0,Yes,No,Yes,Yes,5.118890,33.0,53.0,Alive,Normal (30-60),...,Yes,-99.0,Multiple,8.179584,normal,False,False,False,True,False
2,5.0,No,No,No,No,4.876204,48.0,60.0,Deceased,Normal (30-60),...,No,0.0,Singular,-99.000000,slightly abnormal,False,False,True,True,False
3,13.0,No,Yes,Yes,No,4.687767,25.0,55.0,Alive,-99,...,Yes,-99.0,Singular,6.884071,normal,True,False,True,False,True
4,5.0,No,No,Yes,Yes,5.152362,41.0,38.0,Deceased,Tachypnea,...,No,-99.0,Multiple,6.195178,normal,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285,6.0,Yes,Yes,No,Yes,5.327978,42.0,44.0,Deceased,-99,...,-99,4.0,Singular,4.111201,normal,True,True,True,True,True
9286,5.0,Yes,No,Yes,Yes,4.754449,45.0,27.0,Deceased,Normal (30-60),...,-99,1.0,Singular,-99.000000,inconclusive,True,True,True,True,False
9287,5.0,Yes,Yes,No,Yes,4.824619,26.0,27.0,Alive,Tachypnea,...,No,2.0,Singular,8.291072,-99,True,True,True,True,True
9288,7.0,Yes,No,Yes,No,5.050829,43.0,62.0,Deceased,Normal (30-60),...,-99,0.0,-99,5.180411,slightly abnormal,False,True,False,False,False


In [ ]:
#creating a mapping dictionary
cleanup_nums = {"Genes in mother's side": {"Yes": 1, "No": 0},
                "Inherited from father": {"Yes": 1, "No": 0},
                "Maternal gene": {"Yes": 1, "No": 0},
                "Paternal gene": {"Yes": 1, "No": 0},
                "Status": {"Alive": 1, "Deceased": 0},
                "Folic acid details (peri-conceptional)": {"Yes": 1, "No": 0},
                "H/O serious maternal illness": {"Yes": 1, "No": 0},
                "Assisted conception IVF/ART": {"Yes": 1, "No": 0},
                "History of anomalies in previous pregnancies": {"Yes": 1, "No": 0},
                "Follow-up": {"High": 1, "Low": 0},
               "Place of birth":{"Institute": 1, "Home": 0},
               "Birth defects":{"Singular": 1, "Multiple": 2}}
#replace words with numbers
data = data.replace(cleanup_nums)
data.head()

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),...,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5
0,6.0,0,1,0,0,4.981655,38.0,61.0,1,Tachypnea,...,-99,2.0,2,-99.000000,slightly abnormal,True,True,True,True,True
1,10.0,1,0,1,1,5.118890,33.0,53.0,1,Normal (30-60),...,1,-99.0,2,8.179584,normal,False,False,False,True,False
2,5.0,0,0,0,0,4.876204,48.0,60.0,0,Normal (30-60),...,0,0.0,1,-99.000000,slightly abnormal,False,False,True,True,False
3,13.0,0,1,1,0,4.687767,25.0,55.0,1,-99,...,1,-99.0,1,6.884071,normal,True,False,True,False,True
4,5.0,0,0,1,1,5.152362,41.0,38.0,0,Tachypnea,...,0,-99.0,2,6.195178,normal,True,True,True,True,False


In [ ]:
#one encoding
data = pd.get_dummies(data, columns=["Respiratory Rate (breaths/min)", "Gender", "Birth asphyxia", "Autopsy shows birth defect (if applicable)",
                            "H/O radiation exposure (x-ray)", "H/O substance abuse", "Heart Rate (rates/min"])

data

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Follow-up,...,H/O radiation exposure (x-ray)_Not applicable,H/O radiation exposure (x-ray)_Yes,H/O substance abuse_-,H/O substance abuse_-99,H/O substance abuse_No,H/O substance abuse_Not applicable,H/O substance abuse_Yes,Heart Rate (rates/min_-99,Heart Rate (rates/min_Normal,Heart Rate (rates/min_Tachycardia
0,6.0,0,1,0,0,4.981655,38.0,61.0,1,0,...,0,1,1,0,0,0,0,0,1,0
1,10.0,1,0,1,1,5.118890,33.0,53.0,1,0,...,0,0,0,1,0,0,0,1,0,0
2,5.0,0,0,0,0,4.876204,48.0,60.0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,13.0,0,1,1,0,4.687767,25.0,55.0,1,0,...,0,0,0,1,0,0,0,1,0,0
4,5.0,0,0,1,1,5.152362,41.0,38.0,0,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285,6.0,1,1,0,1,5.327978,42.0,44.0,0,1,...,0,0,1,0,0,0,0,0,1,0
9286,5.0,1,0,1,1,4.754449,45.0,27.0,0,-99,...,1,0,1,0,0,0,0,0,0,1
9287,5.0,1,1,0,1,4.824619,26.0,27.0,1,1,...,0,0,0,0,1,0,0,0,0,1
9288,7.0,1,0,1,0,5.050829,43.0,62.0,0,1,...,1,0,0,0,0,0,1,0,0,1


In [ ]:
#normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_norm = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

data_norm

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Follow-up,...,H/O radiation exposure (x-ray)_Not applicable,H/O radiation exposure (x-ray)_Yes,H/O substance abuse_-,H/O substance abuse_-99,H/O substance abuse_No,H/O substance abuse_Not applicable,H/O substance abuse_Yes,Heart Rate (rates/min_-99,Heart Rate (rates/min_Normal,Heart Rate (rates/min_Tachycardia
0,0.428571,0.0,1.0,0.0,0.0,0.553563,0.606061,0.931818,1.0,0.99,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.714286,1.0,0.0,1.0,1.0,0.641776,0.454545,0.750000,1.0,0.99,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.357143,0.0,0.0,0.0,0.0,0.485780,0.909091,0.909091,0.0,0.99,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.928571,0.0,1.0,1.0,0.0,0.364654,0.212121,0.795455,1.0,0.99,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.357143,0.0,0.0,1.0,1.0,0.663292,0.696970,0.409091,0.0,0.99,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285,0.428571,1.0,1.0,0.0,1.0,0.776176,0.727273,0.545455,0.0,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9286,0.357143,1.0,0.0,1.0,1.0,0.407517,0.818182,0.159091,0.0,0.00,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9287,0.357143,1.0,1.0,0.0,1.0,0.452621,0.242424,0.159091,1.0,1.00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9288,0.500000,1.0,0.0,1.0,0.0,0.598027,0.757576,0.954545,0.0,1.00,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
def predict(X_test, k):
    m = X_train.shape[0]
    n = X_test.shape[0]
    y_pred = []

    # Calculating distances with Euclidean metric
    for i in range(n):
        distances = []
        for j in range(m):
            d = (np.sqrt(np.sum(np.square(X_test[i,:] - X_train[j,:]))))
            distances.append((d, Y_train_1[j])) #for Genetic Disorder
            #distances.append((d, Y_train_2[j])) #for Disorder Subclass
        distances = sorted(distances) #sort in ascending order

        neighbors = []
        for item in range(k):
            neighbors.append(distances[item][1]) #k nearest neighbors
        y_pred.append(stats.mode(neighbors)[0][0]) #prediction
    return y_pred

In [ ]:
#Average error function
def avgErr(pred, actual):
    err = 0
    for i in range(len(pred)):
        if pred[i] != actual[i]:
            err += 1
        else:
            err += 0
    avg_err = (1/(len(pred)))*err
    return avg_err

In [ ]:
#Evaluation
scratch_pred_5 = predict(X_test, 5)
print("Average Error(From Scratch) k = 5: ", avgErr(scratch_pred_5, Y_test_1)*100)

scratch_pred_20 = predict(X_test, 20)
print("Average Error(From Scratch) k = 20: ", avgErr(scratch_pred_20, Y_test_1)*100)

scratch_pred_50 = predict(X_test, 50)
print("Average Error(From Scratch) k = 50: ", avgErr(scratch_pred_50, Y_test_1)*100)

In [ ]:
#K Nearest Neighbors with library
from sklearn.neighbors import KNeighborsClassifier
KNN = KNN = KNeighborsClassifier(n_neighbors = 5, metric='euclidean').fit(X_test, Y_test_1)
lib_pred = KNN.predict(X_test)
print("Average Error(library) k = 5: ", avgErr(lib_pred, Y_test_1)*100)

KNN = KNeighborsClassifier(n_neighbors = 20, metric='euclidean').fit(X_train, Y_train_1)
lib_pred = KNN.predict(X_test)
print("Average Error(library) k = 20: ", avgErr(lib_pred, Y_test_1)*100)

KNN_5 = KNeighborsClassifier(n_neighbors = 50, metric='euclidean').fit(X_train, Y_train_1)
lib_pred = KNN.predict(X_test)
print("Average Error(library) k = 50: ", avgErr(lib_pred, Y_test_1)*100)_